# Reference Manager Citation Errors

12-18-23 KV
2-20-23 KV


In [281]:
# Import needed packages

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

In [282]:
# Read data from where it's entered into CSV

filepath = '/Users/kvrouwenvelder/Library/CloudStorage/OneDrive-AmericanGeophysicalUnion/Documents/Data/Reference-Manager-Work/reference-manager-analysis/'
filename = 'reference-manager-data-v2-'
reference_manager_filenames = ['BibTeX','repo-recommended-citation','Zotero-plugin','Zotero-wizard','BibDesk','Sciwheel-plugin',
                               'SciWheel-wizard','CrossCite','Endnote-plugin','Endnote-wizard','Papers-plugin','Papers-wizard','Paperpile-plugin',
                               'Paperpile-wizard','Mendeley-plugin','Mendeley-wizard','RefWorks-plugin','Refworks-wizard','Export-options']

# import files
bibtex = pd.read_csv(filepath+filename+reference_manager_filenames[0]+'.csv')
repo_recommended_citation = pd.read_csv(filepath+filename+reference_manager_filenames[1]+'.csv')
zotero_plugin = pd.read_csv(filepath+filename+reference_manager_filenames[2]+'.csv')
zotero_wizard = pd.read_csv(filepath+filename+reference_manager_filenames[3]+'.csv')
bib_desk = pd.read_csv(filepath+filename+reference_manager_filenames[4]+'.csv')
sciwheel_plugin = pd.read_csv(filepath+filename+reference_manager_filenames[5]+'.csv')
sciwheel_wizard = pd.read_csv(filepath+filename+reference_manager_filenames[6]+'.csv')
crosscite = pd.read_csv(filepath+filename+reference_manager_filenames[7]+'.csv')
endnote_plugin = pd.read_csv(filepath+filename+reference_manager_filenames[8]+'.csv')
endnote_wizard = pd.read_csv(filepath+filename+reference_manager_filenames[9]+'.csv')
papers_plugin = pd.read_csv(filepath+filename+reference_manager_filenames[10]+'.csv')
papers_wizard = pd.read_csv(filepath+filename+reference_manager_filenames[11]+'.csv')
paperpile_plugin = pd.read_csv(filepath+filename+reference_manager_filenames[12]+'.csv')
paperpile_wizard = pd.read_csv(filepath+filename+reference_manager_filenames[13]+'.csv')
mendeley_plugin = pd.read_csv(filepath+filename+reference_manager_filenames[14]+'.csv')
mendeley_wizard = pd.read_csv(filepath+filename+reference_manager_filenames[15]+'.csv')
refworks_plugin = pd.read_csv(filepath+filename+reference_manager_filenames[16]+'.csv')
refworks_wizard = pd.read_csv(filepath+filename+reference_manager_filenames[17]+'.csv')
export_options = pd.read_csv(filepath+filename+reference_manager_filenames[18]+'.csv')

list_of_ref_managers = [bibtex,repo_recommended_citation,zotero_plugin,zotero_wizard,bib_desk,sciwheel_plugin,
                       sciwheel_wizard,crosscite, endnote_plugin, endnote_wizard, papers_plugin,papers_wizard,paperpile_plugin,
                       paperpile_wizard,mendeley_plugin,mendeley_wizard,refworks_plugin,refworks_wizard]


# Functions

In [283]:
# Set up input fields

# Define export metadata fields for count_fields

ex_cols = ['export-meta-type', 'export-meta-authors','export-meta-title','export-meta-date','export-meta-DOI',
            'export-meta-repo','export-meta-version','export-meta-access-date']

# Define import metadata fields for count_fields
im_cols = ['import-meta-type', 'import-meta-authors','import-meta-title','import-meta-date','import-meta-DOI',
            'import-meta-repo','import-meta-version','import-meta-access-date']


In [284]:
# Count correct, missing, incorrect metadata fields for each repository and reference manager (add cols to df)

def count_fields(input_df,cols,col_type):
    input_df['count_incorrect_'+col_type]=input_df[cols].sum(axis=1, numeric_only=True) # add a column to df with count "incorrect"
    input_df['count_missing_'+col_type]= input_df[cols].isna().sum(axis=1, numeric_only=True) # add a column to df with count 'missing'
    # The count correct is len(cols) - (# incorrect + # missing)
    input_df['count_correct_'+col_type] = len(cols) - (input_df['count_incorrect_'+col_type]+input_df['count_missing_'+col_type])
    return input_df


In [285]:
# Count correct, missing, incorrect across all REFERENCE MANAGERS

def sum_across_repos(list_of_dfs, cols, col_type): 
    # list_of_dfs is df names you wish to analyze; cols is list of columns to analyze; col_type is "ex" or "im"
    
    # init full sums
    one_sum = np.zeros(3)
    full_sums = np.tile(one_sum, (len(list_of_dfs), 1))
    for i in range(len(full_sums)):
        full_sums[i] = np.zeros(3)
    

    iter = 0 #start counter
    
    for df in list_of_dfs: # go through list of dfs
        count_fields(df, cols, col_type)
        sum_incorrect = np.sum(df['count_incorrect_' + col_type]) # this will be the first list element in return
        sum_missing = np.sum(df['count_missing_' + col_type]) # This will be the second list element in return
        sum_correct = np.sum(df['count_correct_' + col_type]) #this will be the 3rd list element in return
        sums = [sum_incorrect, sum_missing, sum_correct]
        full_sums[iter] = sums
        iter += 1
    
    return full_sums
    
# print(full_sums[0][1]) # access an individual element in this list using list[rows][columns]


In [286]:
# Count correct, missing, incorrect across all REPOSITORIES

def sum_across_refmans(list_of_dfs, cols, col_type): 
    # list_of_dfs is df names you wish to analyze; cols is list of columns to analyze; col_type is "ex" or "im"
    
    # init full sums df
    # Number of rows will = # repos in this list
    data = {str('sum_incorrect_'+col_type): np.zeros(len(repo_recommended_citation['Repository'])), #init w zeros
        str('sum_missing_'+col_type): np.zeros(len(repo_recommended_citation['Repository'])),
        str('sum_correct_'+col_type): np.zeros(len(repo_recommended_citation['Repository']))}


    full_sums_df = pd.DataFrame(data) # make empty df 
    full_sums_df['Repository'] = repo_recommended_citation['Repository'] # Add repository column
    
    for df in list_of_dfs: # go through list of dfs
        count_fields(df, cols, col_type) #returns that dataframe with the 3 sums added
        
        #add sum of each new column to existing dataframe
        full_sums_df['sum_incorrect_'+col_type] += df['count_incorrect_'+col_type]
        full_sums_df['sum_missing_'+col_type] += df['count_missing_'+col_type]
        full_sums_df['sum_correct_'+col_type] += df['count_correct_'+col_type]
    
    return full_sums_df
    

In [287]:
# Sum across all import and export fields and return incorrect, missing, and correct sums

def sum_across_fields(list_of_dfs): 
    
    #initialize store_data

    data = {'sum_incorrect': np.zeros(24), # set based on # of columns in endnote_plugin
           'sum_missing': np.zeros(24),
           'sum_correct': np.zeros(24)}

    # run for first dataset in list_of_dfs to set correct indices for dataframe:

    data['sum_incorrect'] = list_of_dfs[0].sum(axis=0, numeric_only=True) # sum incorrect
    data['sum_missing'] = list_of_dfs[0].isna().sum(axis=0, numeric_only=True) # sum missing
    data['sum_correct'].fill(len(list_of_dfs)*len(zotero_plugin['Repository'])) # sum correct; fill with total value possible for all fields! 

    # itera = 0

    # iterate through list of dfs, excluding that first one

    for df in list_of_dfs[1:]:
        data['sum_incorrect'] += df.sum(axis=0, numeric_only=True) # sum incorrect
        data['sum_missing'] += df.isna().sum(axis=0, numeric_only=True) # sum missing

        # itera += 1
        # print(itera) # just to check how far the loop makes it

    # Calculate sum_correct as the total possible fields minus incorrect and missing sums
    data['sum_correct'] = data['sum_correct'] - (data['sum_incorrect']+data['sum_missing'])

    # create dataframe
    store_data = pd.DataFrame(data)

    # Note I checked this output using the export-meta-access-date column. sum_correct here is 25
    # and looking at the source data, this matches the number of "0" codes we recorded!

    return store_data

# Repository recommended citation
* How many repositories offer a recommended citation? 
* How many repositories offer a recommended citation that is correct when compared to DataCite metadata?
* How many repositories define a citation style? 
* How many repositories offer an opportunity to change the citation style to what journal requires?

In [288]:
filename2 = 'repository-recommended-citation'

# Count the correct , incorrect, missing fields - Export Only
count_fields(repo_recommended_citation, ex_cols,'ex')

# How many repositories specify citation style?

num_repo_citation_style = len(repo_recommended_citation[repo_recommended_citation['Citation-style']=='APA'])
print(str(num_repo_citation_style)+' repositories specify a citation style (APA).')

# Create a stacked bar graph with correct, incorrect, missing fields for each repository

## Sort df by most correct fields
repo_recommended_citation_sort = repo_recommended_citation.sort_values(['count_correct_ex'], ascending = False)

## Start graphing
ax = repo_recommended_citation_sort.plot.bar(x = 'Repository', y = ['count_correct_ex','count_missing_ex','count_incorrect_ex'], stacked = True, label = ['Correct', 'Missing','Incorrect'],
                                            color = ['tab:blue','tab:orange','tab:red'])
plt.ylabel('# Metadata Fields')
plt.xticks(rotation = 75)
ax.legend(loc = 'lower left',framealpha=1)

# need to adjust colors, font sizes, rotation of x labels, graph shape, etc.

# Save CSV

repo_recommended_citation_sort.iloc[:,[0,5,6,7,8,9,10,11,12,13,14,15]].to_csv(str(filepath+filename+filename2+'.csv')) # Save CSV of df with graph data

# Save fig
fig.savefig((filepath+filename+filename2+'_vert'+'.png'),dpi=300,bbox_inches='tight')


KeyError: 'Citation-style'

In [289]:
repo_recommended_citation

,Unnamed: 0,Repository,export-meta-type,export-meta-authors,export-meta-title,export-meta-date,export-meta-DOI,export-meta-repo,export-meta-version,export-meta-access-date,count_incorrect_ex,count_missing_ex,count_correct_ex
0,0,Zenodo,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,2,6.0
1,1,Figshare,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,2,6.0
2,3,IEDA,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,2,6.0
3,6,FDSN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,2,6.0
4,10,Dryad,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,2,6.0
5,11,Dataverse,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,2,6.0
6,4,NCAR,NaN,0.0,0.0,0.0,0.0,1.0,0.0,NaN,1.0,2,5.0
7,8,PANGAEA,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,3,5.0
8,9,EDI,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,3,5.0
9,7,Mendeley Data,NaN,1.0,0.0,0.0,0.0,1.0,0.0,NaN,2.0,2,4.0


In [ ]:
# Horizonal bar graph of repository recommended citation

## Sort df by most correct fields
repo_recommended_citation_sort2 = repo_recommended_citation.sort_values(['count_correct_ex'], ascending = True)

# Graph
ax = repo_recommended_citation_sort2.plot.barh(x = 'Repository', y = ['count_correct_ex','count_missing_ex','count_incorrect_ex'], stacked = True, label = ['Correct', 'Missing','Incorrect'],
                                            color = ['tab:blue','tab:orange','tab:red'])
plt.xlabel('# Metadata Fields')
#ax.legend(loc = 'lower left',framealpha=1)
ax.legend(bbox_to_anchor = (1., 1))

plt.show()

# Save fig
fig.savefig((filepath+filename+filename2+'_horiz'+'.png'),dpi=300,bbox_inches='tight')


# Graph correct fields across all repositories
* For each reference manager, what are the # of correct fields across all repositories?

This can be done for either the import or export fields.


In [ ]:
# Filename

filename2 = 'correct-fields-all-repos'

# Must exclude crosscite for import
# Also exclude bib_desk (import used for something different)

list_of_dfs = (zotero_plugin,zotero_wizard,sciwheel_plugin, sciwheel_wizard, 
             endnote_plugin, endnote_wizard, papers_plugin,papers_wizard,paperpile_plugin,paperpile_wizard,mendeley_plugin,
               mendeley_wizard,refworks_plugin,refworks_wizard)

# Get list of reference-manager-filename indices to add to repo-import-sums-df
list_of_indices = reference_manager_filenames[2:4]+reference_manager_filenames[5:7]+reference_manager_filenames[8:-1]



repo_import_sums = sum_across_repos(list_of_dfs, im_cols, 'im')

repo_import_sums_df = pd.DataFrame(data = repo_import_sums, index = list_of_indices, columns = ['sum_incorrect', 'sum_missing', 'sum_correct'])
repo_import_sums_df_sort = repo_import_sums_df.sort_values(by=['sum_correct'],ascending = False)



# Make graph 

fig, ax = plt.subplots()
x = np.arange(len(list_of_dfs)) # of reference managers
y = 14*8 # Max value possible for repo_export_sums for each ref manager. 14 repos, 8 fields per repo

# Get xlabels from index and fix them up
xlabels = repo_import_sums_df_sort.index.str.replace('-',' ') # Use labels from repo_import_sums_df_SORT index and remove hyphen
xlabels = xlabels.str.replace('wizard','App') # change wizard to App
xlabels = xlabels.str.replace('plugin','Plugin') # fix capitalization

ylabels = ['0%','25%','50%','75%','100%']

offset = 0.25
width = 0.25

# to get percentages: df[sum_column]/y*100

# Graph all
ax.bar(x - offset, (repo_import_sums_df_sort['sum_correct']/y*100), width = width, label = 'Correct', color = 'tab:blue')
ax.bar(x + offset, (repo_import_sums_df_sort['sum_incorrect']/y*100), width = width, label = 'Incorrect', color = 'tab:red')
ax.bar(x, (repo_import_sums_df_sort['sum_missing']/y*100), width = width, label = 'Missing', color = 'tab:orange')

# set labels and ticks and axes
ax.set_xticks(x,xlabels, rotation = 75) # Either use repo_import_sums_df_sort.index for labels or new string, 'labels'
ax.set_yticks([0,25,50,75,100],ylabels) # Either [0,y/4,y/2,3*y/4,y] for raw # metadata fields or [0, 25, 50, 100] for %
ax.set_ylabel('% Total Metadata Fields')
ax.legend()

plt.show()

# Save CSV

repo_import_sums_df_sort.to_csv(str(filepath+filename+filename2+'.csv')) # Save CSV of df with graph data

# Save fig

fig.savefig((filepath+filename+filename2+'_vert'+'.png'),dpi=300,bbox_inches='tight')


In [ ]:
# Horizontal version

# Sort ascending = True to put correct values at top
repo_import_sums_df_sort = repo_import_sums_df.sort_values(by=['sum_correct'],ascending = True)

# Make graph 

fig, ax = plt.subplots()
x = np.arange(len(list_of_dfs)) # of reference managers
y = 14*8 # Max value possible for repo_export_sums for each ref manager. 14 repos, 8 fields per repo

# Get xlabels from index and fix them up
xlabels = repo_import_sums_df_sort.index.str.replace('-',' ') # Use labels from repo_import_sums_df_SORT index and remove hyphen
xlabels = xlabels.str.replace('wizard','[App]') # change wizard to App
xlabels = xlabels.str.replace('plugin','[Plugin]') # fix capitalization

ylabels = ['0%','25%','50%','75%','100%']

offset = 0.25
width = 0.25

# to get percentages: df[sum_column]/y*100

# Graph all
ax.barh(x + offset, (repo_import_sums_df_sort['sum_correct']/y*100), width, label = 'Correct', color = 'tab:blue')
ax.barh(x - offset, (repo_import_sums_df_sort['sum_incorrect']/y*100),width, label = 'Incorrect', color = 'tab:red')
ax.barh(x, (repo_import_sums_df_sort['sum_missing']/y*100), width,label = 'Missing', color = 'tab:orange')

# set labels and ticks and axes
ax.set_yticks(x,xlabels, rotation = 0) # Either use repo_import_sums_df_sort.index for labels or new string, 'labels'
ax.set_xticks([0,25,50,75,100],ylabels) # Either [0,y/4,y/2,3*y/4,y] for raw # metadata fields or [0, 25, 50, 100] for %
ax.set_xlabel('% Total Metadata Fields')
ax.legend(bbox_to_anchor=(0.75, 0.6))

plt.show()

# Save fig

fig.savefig((filepath+filename+filename2+'_horiz'+'.png'),dpi=300,bbox_inches='tight')


# Bibtex
* How many repositories offer Bibtex exports and in what formats?
* Are Bibtex fields correct? 

In [ ]:
# First how many repos offer Bibtex exports

# filename
filename2 = 'bibtex'
# Count whether there is a bibtex export or not

# no bibtex
sum_no_bibtex = np.sum(bibtex['export-exists'].isna())
sum_bibtex = len(bibtex['export-exists'])-sum_no_bibtex

# Filter by type
bibtex_na = bibtex[bibtex['export-exists'].isna()]
bibtex_dataset = bibtex[bibtex['export-file-type']=='dataset']
bibtex_misc = bibtex[bibtex['export-file-type'].str.contains('misc', na=False)]
bibtex_data = bibtex[bibtex['export-file-type']=='data']
bibtex_incoll = bibtex[bibtex['export-file-type'].str.contains('incollection', na=False)]
bibtex_article = bibtex[bibtex['export-file-type'].str.contains('article', na=False)]


# # bar chart for bibtex or NO
# fig, ax = plt.subplots()
# ax.bar([1,2], [sum_bibtex, sum_no_bibtex], label = 'Correct')

labels = ['No Export','@dataset','@misc','@data','@InCollection','@article']
sizes = [len(bibtex_na['export-exists']), len(bibtex_dataset['export-exists']),len(bibtex_misc['export-exists']),
         len(bibtex_data['export-exists']),len(bibtex_incoll['export-exists']),len(bibtex_article['export-exists'])]
colors = ['tab:red','tab:blue','tab:orange','tab:blue','tab:grey','tab:orange'] # colors corresp. to correctness

# Plot pie chart
fig1, ax1 = plt.subplots()
patches, texts, autotexts = ax1.pie(sizes, labels=labels, autopct='%1.1f%%', colors = colors)

# set inside label parameters: color, size, weight
# Color = white
for autotext in autotexts: 
    autotext.set_color('white')

# weight and size
plt.setp(autotexts, size=11, weight = 'bold')
    
# show graph
plt.show()

# Save fig

fig.savefig((filepath+filename+filename2+'_pie'+'.png'),dpi=300,bbox_inches='tight')



In [ ]:
# Filename

filename2 = 'bibtex-fields-by-repo'

# Count the correct , incorrect, missing fields - Import Only
count_fields(bibtex, im_cols,'im')


# Sort fields (filter by only repos offering bibtex export and sort by # correct)
bibtex_sort = bibtex.loc[bibtex['export-exists']==0.0].sort_values(by=['count_correct_im'],ascending = False)

# Graph
ax2 = bibtex_sort.plot.bar(x = 'Repository', y = ['count_correct_im','count_missing_im','count_incorrect_im'], 
                           stacked = True, 
                           label = ['Correct', 'Missing','Incorrect'],
                          color = ['tab:blue','tab:orange','tab:red'])
plt.ylabel('# Metadata Fields')

# Get x labels
ax2.legend(loc = 'lower left',framealpha=1)
ax2.set_xticks(np.arange(7), labels = bibtex_sort['Repository'],rotation=75)
plt.show()

# Save CSV

bibtex_sort.to_csv(str(filepath+filename+filename2+'.csv')) # Save CSV of df with graph data

# Save fig
fig.savefig((filepath+filename+filename2+'_bar'+'.png'),dpi=300,bbox_inches='tight')


In [ ]:
# Horizontal version

# Sort fields (filter by only repos offering bibtex export and sort by # correct)
bibtex_sort2 = bibtex.loc[bibtex['export-exists']==0.0].sort_values(by=['count_correct_im'],ascending = True)

# Graph
ax2 = bibtex_sort2.plot.barh(x = 'Repository', y = ['count_correct_im','count_missing_im','count_incorrect_im'], 
                           stacked = True, 
                           label = ['Correct', 'Missing','Incorrect'],
                          color = ['tab:blue','tab:orange','tab:red'])
plt.xlabel('# Metadata Fields')

# Get x labels
ax2.legend(bbox_to_anchor = (1., 1))

plt.show()

# Save fig

fig.savefig((filepath+filename+filename2+'_bar_horiz'+'.png'),dpi=300,bbox_inches='tight')


# Best Repository Citation Metadata

* are there any repositories that are performing exceptionally well across reference managers?

Use import data and sum across sheets - how many '0s'. 


Could also break out the 8 metadata fields on a by-repository basis and count across the sheets for a single repository for each field, but this would be pretty challenging to graph

In [ ]:
# Filename

filename2 = 'repo-metadata-across-refmans'

# Must exclude crosscite for import
# Bibdesk is also excluded

list_of_dfs = (zotero_plugin,zotero_wizard,bib_desk,sciwheel_plugin, sciwheel_wizard, 
               endnote_wizard, endnote_plugin, papers_plugin,papers_wizard,paperpile_plugin,paperpile_wizard,mendeley_plugin,
               mendeley_wizard,refworks_wizard, refworks_plugin)

# Get repository metadata field sums across reference managers
refman_import_sums = sum_across_refmans(list_of_dfs, im_cols, 'im')


# Set labels
ylabels = ['0%','25%','50%','75%','100%']

# Sort values
refman_import_sums_sort = refman_import_sums.sort_values(by=['sum_correct_im'],ascending = False)

# Make graph
fig, ax = plt.subplots()
x = np.arange(14)

y = len(list_of_dfs)*8 # Max value possible for refman_sums for each ref manager

# Percents are df[sum_column]/y*100

offset = 0.25
width = 0.25
ax.bar(x - offset, (refman_import_sums_sort['sum_correct_im']/y*100), width = width, label = 'Correct', color = 'tab:blue')
ax.bar(x + offset, (refman_import_sums_sort['sum_incorrect_im']/y*100), width = width, label = 'Incorrect', color = 'tab:red')
ax.bar(x, (refman_import_sums_sort['sum_missing_im']/y*100), width = width, label = 'Missing', color = 'tab:orange')

# set labels and ticks and axes
ax.set_xticks(x,refman_import_sums_sort['Repository'], rotation = 75) # Use Repository column
ax.set_yticks([0,(y/y)/4*100,(y/y)/2*100,3*(y/y)/4*100,y/y*100],ylabels)
ax.set_ylabel('% Total Metadata Fields')
ax.legend()

plt.show()

# Save CSV

refman_import_sums.to_csv(str(filepath+filename+filename2+'.csv')) # Save CSV of df with graph data

# Save fig
fig.savefig((filepath+filename+filename2+'_vert'+'.png'),dpi=300,bbox_inches='tight')


In [ ]:
# Horizontal version

# Sort values
refman_import_sums_sort2 = refman_import_sums.sort_values(by=['sum_correct_im'],ascending = True)

# Make graph
fig, ax = plt.subplots()
x = np.arange(14)

y = len(list_of_dfs)*8 # Max value possible for refman_sums for each ref manager

# Percents are df[sum_column]/y*100

offset = 0.25
width = 0.25
ax.barh(x + offset, (refman_import_sums_sort2['sum_correct_im']/y*100), width, label = 'Correct', color = 'tab:blue')
ax.barh(x - offset, (refman_import_sums_sort2['sum_incorrect_im']/y*100), width, label = 'Incorrect', color = 'tab:red')
ax.barh(x, (refman_import_sums_sort2['sum_missing_im']/y*100), width, label = 'Missing', color = 'tab:orange')

# set labels and ticks and axes
ax.set_yticks(x,refman_import_sums_sort2['Repository']) # Use Repository column
ax.set_xticks([0,(y/y)/4*100,(y/y)/2*100,3*(y/y)/4*100,y/y*100],ylabels)
ax.set_xlabel('% Total Metadata Fields')
ax.legend(bbox_to_anchor = (1.,1))

plt.show()

# Save fig

fig.savefig((filepath+filename+filename2+'_horiz'+'.png'),dpi=300,bbox_inches='tight')


# Most common errors by metadata field

* by import
* by export



In [ ]:
# set filename

filename2 = 'errors-by-field'

# list dfs to include

# should NOT include cross cite? 
# should ALSO not include BibDesk as import type field is used for something else

list_of_dfs = (zotero_plugin,zotero_wizard,sciwheel_plugin, sciwheel_wizard, endnote_wizard, endnote_plugin, 
               papers_plugin,papers_wizard,paperpile_plugin,paperpile_wizard,mendeley_plugin,
               mendeley_wizard,refworks_wizard, refworks_plugin)

# Get df with sum incorrect, missing, and correct for each metadata field by import and export

store_data = sum_across_fields(list_of_dfs)

# For both export and import, graph as a stack correct, incorrect, missing for each metadata field

# Common parameters for graphing

x = np.arange(8) # of metadata fields
y = len(list_of_dfs)*len(zotero_plugin['Repository']) # total possible correct fields, len list of dfs * 14 repositories

## Start graphing IMPORT

# first sort import
import_sort = store_data.loc['import-meta-DOI':'import-meta-version'].sort_values(by = 'sum_correct',ascending = False)

fig, ax = plt.subplots()

offset = 0.25
width = 0.25
ax.bar(x - offset, (import_sort['sum_correct']/y*100), width = width, label = 'Correct', color = 'tab:blue')
ax.bar(x + offset, (import_sort['sum_incorrect']/y*100), width = width, label = 'Incorrect', color = 'tab:red')
ax.bar(x, (import_sort['sum_missing']/y*100), width = width, label = 'Missing', color = 'tab:orange')

# Get labels and clean them up
xlabels = import_sort.index #labels will be import_sort.index Prev store_data index DOI thru version. Import is 16:24
xlabels = xlabels.str.replace('-',' ') # replace hyphen
xlabels = xlabels.str.replace('import','') #don't include 'import'
xlabels = xlabels.str.replace('meta','') #don't include 'meta'
xlabels = xlabels.str.replace('repo','repository') 

# set labels and ticks and axes

ax.set_xticks(x,xlabels, rotation = 75) # Use xlabels or index
ax.set_yticks([0,(y/y)/4*100,(y/y)/2*100,3*(y/y)/4*100,y/y*100],ylabels)
ax.set_ylabel('% Total Metadata Fields')
ax.legend()
plt.title('Import')

plt.show()

# Save fig

fig.savefig((filepath+filename+filename2+'_import_vert'+'.png'),dpi=300,bbox_inches='tight')

## Start graphing EXPORT 

# first sort export
export_sort = store_data.loc['export-meta-DOI':'export-meta-version'].sort_values(by = 'sum_correct',ascending = False)

fig, ax = plt.subplots()

offset = 0.25
width = 0.25
ax.bar(x - offset, (export_sort['sum_correct']/y*100), width = width, label = 'Correct', color = 'tab:blue')
ax.bar(x + offset, (export_sort['sum_incorrect']/y*100), width = width, label = 'Incorrect', color = 'tab:red')
ax.bar(x, (export_sort['sum_missing']/y*100), width = width, label = 'Missing', color = 'tab:orange')

# Get labels and clean them up
xlabels = export_sort.index #labels will be export_sort index Prev store_data index DOI thru version. Export is 7:15
xlabels = xlabels.str.replace('-',' ') # replace hyphen
xlabels = xlabels.str.replace('export','') #don't include 'import'
xlabels = xlabels.str.replace('meta','') #don't include 'meta'
xlabels = xlabels.str.replace('repo','repository') 


# set labels and ticks and axes
ax.set_xticks(x,xlabels, rotation = 75) # Use xlabels or index
ax.set_yticks([0,(y/y)/4*100,(y/y)/2*100,3*(y/y)/4*100,y/y*100],ylabels)
ax.set_ylabel('% Total Metadata Fields')
ax.legend()
plt.title('Export')

plt.show()

# Save fig

fig.savefig((filepath+filename+filename2+'_export_vert'+'.png'),dpi=300,bbox_inches='tight')

# Save CSV for both

store_data.to_csv(str(filepath+filename+filename2+'.csv')) # Save CSV of df with graph data


In [ ]:
# Horizontal version

# Common parameters for graphing

x = np.arange(8) # of metadata fields
y = len(list_of_dfs)*len(zotero_plugin['Repository']) # total possible correct fields, len list of dfs * 14 repositories

## Start graphing IMPORT 

# first sort import
import_sort = store_data.loc['import-meta-DOI':'import-meta-version'].sort_values(by = 'sum_correct',ascending = True)

fig, ax = plt.subplots()

offset = 0.25
width = 0.25
ax.barh(x + offset, import_sort['sum_correct']/y*100, width, label = 'Correct', color = 'tab:blue')
ax.barh(x - offset, import_sort['sum_incorrect']/y*100, width, label = 'Incorrect', color = 'tab:red')
ax.barh(x, import_sort['sum_missing']/y*100, width, label = 'Missing', color = 'tab:orange')

# Get labels and clean them up
xlabels = import_sort.index #labels will be import_sort index DOI thru version. Prev used store_data index, import is 16:24
xlabels = xlabels.str.replace('-',' ') # replace hyphen
xlabels = xlabels.str.replace('import','') #don't include 'import'
xlabels = xlabels.str.replace('meta','') #don't include 'meta'
xlabels = xlabels.str.replace('repo','repository') 

# set labels and ticks and axes
ax.set_yticks(x,xlabels) # Use xlabels or index
ax.set_xticks([0,(y/y)/4*100,(y/y)/2*100,3*(y/y)/4*100,y/y*100],ylabels)
ax.set_xlabel('% Total Metadata Fields')
ax.legend()
plt.title('Import')

plt.show()

# Save fig

fig.savefig((filepath+filename+filename2+'_import_horiz'+'.png'),dpi=300,bbox_inches='tight')


## Start graphing EXPORT 

# first sort export
export_sort = store_data.loc['export-meta-DOI':'export-meta-version'].sort_values(by = 'sum_correct',ascending = True)

fig, ax = plt.subplots()

offset = 0.25
width = 0.25
ax.barh(x + offset, (export_sort['sum_correct']/y*100), width, label = 'Correct', color = 'tab:blue')
ax.barh(x - offset, (export_sort['sum_incorrect']/y*100), width, label = 'Incorrect', color = 'tab:red')
ax.barh(x, (export_sort['sum_missing']/y*100), width, label = 'Missing', color = 'tab:orange')

# Get labels and clean them up
xlabels = export_sort.index #labels will be export_sort index DOI thru version. Prev used store_data index, export
xlabels = xlabels.str.replace('-',' ') # replace hyphen
xlabels = xlabels.str.replace('export','') #don't include 'import'
xlabels = xlabels.str.replace('meta','') #don't include 'meta'
xlabels = xlabels.str.replace('repo','repository') 

# set labels and ticks and axes
ax.set_yticks(x,xlabels) # Use xlabels or index
ax.set_xticks([0,(y/y)/4*100,(y/y)/2*100,3*(y/y)/4*100,y/y*100],ylabels)
ax.set_xlabel('% Total Metadata Fields')
ax.legend()
plt.title('Export')

plt.show()

# Save fig
fig.savefig((filepath+filename+filename2+'_export_horiz'+'.png'),dpi=300,bbox_inches='tight')
